In [921]:
import pandas as pd
import re

In [922]:
data=pd.read_csv("Tweets.csv",nrows=10000)

In [923]:
data.head()
data.shape
data.dtypes
data["text"].astype(str)

0                     @VirginAmerica What @dhepburn said.
1       @VirginAmerica plus you've added commercials t...
2       @VirginAmerica I didn't today... Must mean I n...
3       @VirginAmerica it's really aggressive to blast...
4       @VirginAmerica and it's a really big bad thing...
                              ...                        
9995    @USAirways @AmericanAir seriously with all the...
9996    @USAirways Flight 4210 been sitting @Bradley_A...
9997    @USAirways new F/As from DFW to CLT this morni...
9998    @USAirways why have me board a plane knowing t...
9999    @USAirways with this livery back in the day. h...
Name: text, Length: 10000, dtype: object

In [924]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [925]:
#lets check percentage of missing values 
data.isnull().sum()*100/len(data)
#we want to perform sentiment analysis thus we only want to keep the columns airline_sentiment and text ,
#and do preprocessing on the text column

tweet_id                         0.00
airline_sentiment                0.00
airline_sentiment_confidence     0.00
negativereason                  42.41
negativereason_confidence       32.62
airline                          0.00
airline_sentiment_gold          99.77
name                             0.00
negativereason_gold             99.81
retweet_count                    0.00
text                             0.00
tweet_coord                     92.66
tweet_created                    0.00
tweet_location                  31.60
user_timezone                   31.81
dtype: float64

In [926]:
#we will drop all columns except airline_sentiment ,airline_sentiment_confidence  and text 

data.drop(["tweet_id",'airline_sentiment_gold',"negativereason","negativereason_confidence","airline","airline_sentiment_gold","name","negativereason_gold","retweet_count",'negativereason_gold','tweet_coord',"tweet_created","tweet_location","user_timezone"],axis=1,inplace=True)

In [927]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,@VirginAmerica What @dhepburn said.
1,positive,0.3486,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,@VirginAmerica and it's a really big bad thing...


In [928]:
#lets check for duplicated rows
data.duplicated().sum()

13

In [929]:
#perfect no duplicates 

In [930]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,@VirginAmerica What @dhepburn said.
1,positive,0.3486,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,@VirginAmerica and it's a really big bad thing...


In [931]:
#convert to lowercase
data["text"]=data["text"].str.lower()


In [932]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,@virginamerica what @dhepburn said.
1,positive,0.3486,@virginamerica plus you've added commercials t...
2,neutral,0.6837,@virginamerica i didn't today... must mean i n...
3,negative,1.0000,@virginamerica it's really aggressive to blast...
4,negative,1.0000,@virginamerica and it's a really big bad thing...


In [933]:
data["text"][21]

'@virginamerica i love this graphic. http://t.co/ut5grrwaaa'

In [934]:
#lets remove urls 
#define regex pattern
#url_pattern=re.compile(r'https?://\s+')
#define function to remove urls from text
def remove_urls(text):
    url_pattern = re.compile('https?://\S+|www\.\S+')
    
    return url_pattern.sub("",text)

#apply function to column "text"
data["text"]=data["text"].apply(remove_urls)



In [935]:
data["text"][21]

'@virginamerica i love this graphic. '

In [936]:
data["text"][18]

'i ❤️ flying @virginamerica. ☺️👍'

In [937]:

#LEZOS: "grinning face" -> happy
#LEZOS: "face with tongue" -> kidding
#LEZOS: "thumbs up" -> approve
emoji_dict = {
    "😀": "happy",
    "😃": "very happy",
    "😄": "laughing happily",
    "😁": "smiling",
    "😆": "laughing",
    "😅": "laughing ironicly",
    "😂": "crying in laughter",
    "🤣": "rolling on the floor laughing",
    "😊": "smiling face with smiling eyes",
    "😇": "smiling face with halo",
    "🙂": "slightly smiling face",
    "🙃": "upside-down face",
    "😉": "winking face",
    "😌": "relieved face",
    "😍": "smiling face with heart-eyes",
    "😘": "face blowing a kiss",
    "😗": "kissing face",
    "😙": "kissing face with smiling eyes",
    "😚": "kissing face with closed eyes",
    "😋": "face savoring food",
    "😛": "face with tongue",
    "😜": "winking face with tongue",
    "😝": "squinting face with tongue",
    "🤑": "money-mouth face",
    "🤗": "hugging face",
    "🤓": "nerd face",
    "😎": "smiling face with sunglasses",
    "🤡": "clown face",
    "🤠": "cowboy hat face",
    "😏": "smirking face",
    "😒": "unamused face",
    "😞": "disappointed face",
    "😔": "pensive face",
    "😟": "worried face",
    "😕": "confused face",
    "🙁": "slightly frowning face",
    "☹️": "frowning face",
    "😣": "persevering face",
    "😖": "confounded face",
    "😫": "tired face",
    "😩": "weary face",
    "😤": "face with steam from nose",
    "😠": "angry face",
    "😡": "pouting face",
    "😶": "face without mouth",
    "😐": "neutral face",
    "😑": "expressionless face",
    "😯": "hushed face",
    "😦": "frowning face with open mouth",
    "😧": "anguished face",
    "😮": "face with open mouth",
    "😲": "astonished face",
    "😵": "dizzy face",
    "😳": "flushed face",
    "😱": "face screaming in fear",
    "😨": "fearful face",
    "😰": "anxious face with sweat",
    "😢": "crying face",
    "😥": "sad but relieved face",
    "🤤": "drooling face",
    "😭": "loudly crying face",
    "😓": "downcast face with sweat",
    "😪": "sleepy face",
    "😴": "sleeping face",
    "🙄": "face with rolling eyes",
    "🤔": "thinking face",
    "🤥": "lying face",
    "😬": "grimacing face",
    "🤐": "zipper-mouth face",
    "🤢": "nauseated face",
    "🤧": "sneezing face",
    "😷": "face with medical mask",
    "🤒": "face with thermometer",
    "🤕": "face with head-bandage",
    "🤢": "nauseated face",
    "🤮": "face vomiting",
    "🤠": "cowboy hat face",
    "🤡": "clown face",
    "👹": "ogre",
    "👺": "goblin",
    "💩": "pile of poo",
    "👻": "ghost",
    "💀": "skull",
    "☠️": "skull and crossbones",
    "👽": "alien",
    "👾": "alien monster",
    "🤖": "robot",
    "🎃": "jack-o-lantern",
    "😺": "grinning cat face",
    "😸": "grinning cat face with smiling eyes",
    "😹": "cat face with tears of joy",
    "👍": "thumps up",
    "❤️": "love"}
def replace_emojis(text):
    # Create a regular expression pattern to match emojis
    emoji_pattern = re.compile("|".join(re.escape(emoji) for emoji in emoji_dict.keys()))
    
    # Replace emojis with their text representations
    return emoji_pattern.sub(lambda x: emoji_dict[x.group()], text)

# Apply the function to the 'text' column
data['text'] = data['text'].apply(replace_emojis)


In [938]:
data["text"][15]

'@virginamerica sfo-pdx schedule is still mia.'

In [939]:

## import re
#lets remove special characters etc
data["text"] = data["text"].str.replace(r"[^\w\s]","", regex=True)
data["text"] = data["text"].str.replace(r"[^\w\s]","", regex=True)


In [940]:
data.head()





,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,virginamerica what dhepburn said
1,positive,0.3486,virginamerica plus youve added commercials to ...
2,neutral,0.6837,virginamerica i didnt today must mean i need t...
3,negative,1.0000,virginamerica its really aggressive to blast o...
4,negative,1.0000,virginamerica and its a really big bad thing a...


In [941]:
#next day

In [942]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,virginamerica what dhepburn said
1,positive,0.3486,virginamerica plus youve added commercials to ...
2,neutral,0.6837,virginamerica i didnt today must mean i need t...
3,negative,1.0000,virginamerica its really aggressive to blast o...
4,negative,1.0000,virginamerica and its a really big bad thing a...


In [943]:
#we wont remove stopwords because If we are training a model for senUment analysis, we might not wish to remove the stop words.
#Movie review: “The movie was not good at all.”
#Text a.er removal of stop words: “movie good”

In [944]:

#lets perfform Expanding contractions
import contractions

def expand_contractions(text):
    return contractions.fix(text)

data["text"] = data["text"].apply(expand_contractions)

In [945]:
#if we perform that it will not run for ages 
#lets perform text correction
#for installation
#!pip install textblob      

#to import
#import textblob            
#from textblob import TextBlob

In [946]:
#data['text'] = data['text'].apply(lambda x: str(TextBlob(x).correct()))

In [947]:
#so far in our text column we have performed  convert to lowercase ,
#remove specific character ,Expanding contractions

In [948]:
#from nltk import word_tokenize, sent_tokenize
#data["text"]=data["text"].apply(word_tokenize)

In [949]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text
0,neutral,1.0000,virginamerica what dhepburn said
1,positive,0.3486,virginamerica plus you have added commercials ...
2,neutral,0.6837,virginamerica i did not today must mean i need...
3,negative,1.0000,virginamerica its really aggressive to blast o...
4,negative,1.0000,virginamerica and its a really big bad thing a...


In [950]:

#lets perform 
data['text']=data['text'].apply(str)
import nltk
from nltk.stem import PorterStemmer
# create an object of class PorterStemmer


stemmer = PorterStemmer()

def stemming(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)


data['stemming_text'] = data['text'].apply(stemming)

In [951]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text,stemming_text
0,neutral,1.0000,virginamerica what dhepburn said,virginamerica what dhepburn said
1,positive,0.3486,virginamerica plus you have added commercials ...,virginamerica plu you have ad commerci to the ...
2,neutral,0.6837,virginamerica i did not today must mean i need...,virginamerica i did not today must mean i need...
3,negative,1.0000,virginamerica its really aggressive to blast o...,virginamerica it realli aggress to blast obnox...
4,negative,1.0000,virginamerica and its a really big bad thing a...,virginamerica and it a realli big bad thing ab...


In [952]:
data["text"][3]


'virginamerica its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse'

In [953]:
data["stemming_text"][3]

'virginamerica it realli aggress to blast obnoxi entertain in your guest face amp they have littl recours'

In [954]:
#we want to compare what will give better results stemming or lemmatizing , i will create a new column that insted
#of stemming we performed lemmatizing on the text 

In [955]:
#lets create a lematizzing function 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [956]:
data['text_lemmatized'] = data.text.apply(lemmatize_text)

In [957]:
data["text"][3]

'virginamerica its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse'

In [958]:
data["text_lemmatized"][3]

['virginamerica',
 'it',
 'really',
 'aggressive',
 'to',
 'blast',
 'obnoxious',
 'entertainment',
 'in',
 'your',
 'guest',
 'face',
 'amp',
 'they',
 'have',
 'little',
 'recourse']

In [959]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text,stemming_text,text_lemmatized
0,neutral,1.0000,virginamerica what dhepburn said,virginamerica what dhepburn said,"[virginamerica, what, dhepburn, said]"
1,positive,0.3486,virginamerica plus you have added commercials ...,virginamerica plu you have ad commerci to the ...,"[virginamerica, plus, you, have, added, commer..."
2,neutral,0.6837,virginamerica i did not today must mean i need...,virginamerica i did not today must mean i need...,"[virginamerica, i, did, not, today, must, mean..."
3,negative,1.0000,virginamerica its really aggressive to blast o...,virginamerica it realli aggress to blast obnox...,"[virginamerica, it, really, aggressive, to, bl..."
4,negative,1.0000,virginamerica and its a really big bad thing a...,virginamerica and it a realli big bad thing ab...,"[virginamerica, and, it, a, really, big, bad, ..."


In [960]:
#lets perform tokenizing in columns "text" and stemming_text
from nltk import word_tokenize, sent_tokenize
data["text"]=data["text"].apply(word_tokenize)
data["stemming_text"]=data["stemming_text"].apply(word_tokenize)

In [961]:
data.head()

,airline_sentiment,airline_sentiment_confidence,text,stemming_text,text_lemmatized
0,neutral,1.0000,"[virginamerica, what, dhepburn, said]","[virginamerica, what, dhepburn, said]","[virginamerica, what, dhepburn, said]"
1,positive,0.3486,"[virginamerica, plus, you, have, added, commer...","[virginamerica, plu, you, have, ad, commerci, ...","[virginamerica, plus, you, have, added, commer..."
2,neutral,0.6837,"[virginamerica, i, did, not, today, must, mean...","[virginamerica, i, did, not, today, must, mean...","[virginamerica, i, did, not, today, must, mean..."
3,negative,1.0000,"[virginamerica, its, really, aggressive, to, b...","[virginamerica, it, realli, aggress, to, blast...","[virginamerica, it, really, aggressive, to, bl..."
4,negative,1.0000,"[virginamerica, and, its, a, really, big, bad,...","[virginamerica, and, it, a, realli, big, bad, ...","[virginamerica, and, it, a, really, big, bad, ..."
